In [1]:
import sys
sys.path.append('../../build')
sys.path.append('../')
import IPSModule as ips
import numpy as np
import matplotlib.pyplot as plt
import lattpy as lp

In [2]:
def compute_vacf(velocities, lags):
    """
    Compute the global Velocity Auto-Correlation Function (VACF).
    For each time lag in 'lags', use all possible starting times to calculate 
    the average velocity dot product, forming a complete VACF curve.
    
    Parameters:
        velocities: Array of shape (n_steps, n_particles, d)
        lags: 1D array of lag values (in number of time steps)
    Returns:
        vacf: 1D array of VACF values corresponding to each lag in 'lags'
    """
    n_steps, n_particles, d = velocities.shape
    vacf = np.zeros(len(lags))
    
    for i, lag in enumerate(lags):
        v1 = velocities[:n_steps - lag]
        v2 = velocities[lag:]
        dot_products = np.sum(v1 * v2, axis=2)  # shape: (n_steps-lag, n_particles)
        vacf[i] = np.mean(dot_products)
    return vacf

def plot_simulation_results(vacf_data, freq_data, method_name):
    """
    Plot the VACF curves and vibrational density of states for simulations.

    Parameters:
        vacf_data: List of tuples (lags, normalized VACF) for each experiment
        freq_data: List of tuples (freq, S(ω)) for each experiment
        method_name: String indicating the simulation method (e.g., 'Langevin', 'NoseHoover')
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Plot VACF curves
    for i, (lags, vacf_norm) in enumerate(vacf_data):
        axes[0].plot(lags, vacf_norm, label=f'VACF {i+1}')
    axes[0].set_xlabel("Lag time")
    axes[0].set_ylabel("Normalized VACF")
    axes[0].set_title(f"{method_name}: Multiple VACF Curves")
    axes[0].legend()

    # Plot vibrational density of states
    for i, (freq, S_omega) in enumerate(freq_data):
        axes[1].plot(freq, S_omega, label=f'Spectrum {i+1}')
    axes[1].set_xlabel("Frequency")
    axes[1].set_ylabel("S(ω)")
    axes[1].set_title(f"{method_name}: Multiple Vibrational Density of States")
    axes[1].legend()

    plt.tight_layout()
    plt.show()

In [3]:
from utils import generate_circle_cluster, SimulationVisualizer
from functools import partial

def run_simulation(particle_info, sim_method_params, sim_params):
    """
    Run multiple simulation experiments for a specified thermostat method and calculate the VACF and vibrational spectrum for each experiment.
    
    Parameters:
        particle_info: Dictionary containing:
                'num_particles': Number of particles
                'init_positions': Initial positions array, shape (num_particles, 2)
        sim_method_params: Dictionary containing parameters for creating the simulation system. For example:
            For the Langevin method:
                {
                  "method": "Langevin",
                  "gamma": 1.0,
                  "temperature": 0.3
                }
            For the NoseHoover method:
                {
                  "method": "NoseHoover",
                  "gamma": 1.0,
                  "temperature": 0.3,
                  "Q": 1.0,
                  "eta": 0.0
                }
        sim_params: Dictionary containing other experiment parameters, such as:
            {
              "num_experiments": 5,
              "num_step": 200000,
              "dt": 0.001,
              "draw_interval": None,   # If None, defaults to num_step // 50
              "epsilon": 1.0,
              "sigma": 1.0,
              "rad": 40.0,
              "vacf_lags": np.arange(0, 200, 2)
            }
    Returns:
        experiments_vacf: List where each element is a tuple (lags, normalized VACF)
        experiments_freq: List where each element is a tuple (freq, S(ω)) containing only the positive frequency part
    """
    from functools import partial
    from utils import SimulationVisualizer

    num_experiments = sim_params.get("num_experiments", 5)
    num_step = sim_params.get("num_step", 200000)
    dt = sim_params.get("dt", 0.001)
    draw_interval = sim_params.get("draw_interval", num_step // 50)
    simulation_steps = sim_params.get("simulation_steps", 100)
    epsilon = sim_params.get("epsilon", 1.0)
    sigma = sim_params.get("sigma", 1.0)
    rad = sim_params.get("rad", 40.0)
    lags = sim_params.get("vacf_lags", np.arange(0, 200, 2))
    show = sim_params.get("show", False)

    start_recording = sim_params.get("start_recording", 0)
    end_recording = sim_params.get("end_recording", num_step-1)
    
    # Simulation method parameters
    method = sim_method_params.get("method", "Langevin")
    gamma = sim_method_params.get("gamma", 1.0)
    temperature = sim_method_params.get("temperature", 0.3)
    # NoseHoover-specific parameters
    Q = sim_method_params.get("Q", 1.0)
    eta = sim_method_params.get("eta", 0.0)
    
    experiments_vacf = []
    experiments_freq = []
    
    for exp in range(num_experiments):
        # Get initial particle information: if particle_info is callable, call it; otherwise, use it directly
        
        num_particles = particle_info['num_particles']
        init_particles_positions = particle_info['init_positions']
        
        # Create the corresponding simulation system based on the method
        if method == 'Langevin':
            p = ips.LangevinSystem(num_particles, gamma, temperature)
        elif method == 'NoseHoover':
            p = ips.NoseHooverLangevinSystem(num_particles, gamma, temperature, Q, eta)
        else:
            raise ValueError("Unknown method: choose 'Langevin' or 'NoseHoover'")
        
        # Set initial positions and initial velocities to 0
        for i in range(num_particles):
            for d in range(2):
                p.get_positions()[d][i] = init_particles_positions[i][d]
                p.get_velocities()[d][i] = 0.0
        
        pair_force_config = {
            "type": "LennardJones",
            "eps": epsilon,
            "sigma": sigma
        }
        confinement_config = {
            "type": "Radial",
            "rad": rad
        }
        if method == 'Langevin':
            simulator = ips.IPS_Simulator_Langevin(p)
        else:
            simulator = ips.IPS_Simulator_NoseHooverLangevin(p)
        simulator.init(pair_force_config, confinement_config)
        
        # List to record velocities
        velocities_recorded = []
        def record_velocities(simulator, p, step, start_recording, end_recording):
            if step >= start_recording and step <= end_recording:
                velocities_recorded.append(np.copy(p.get_velocities()))
        
        if exp == num_experiments - 1 and show:
            sim_visualizer = SimulationVisualizer(simulator=simulator, particle_system=p, rad=rad, 
                                                  draw_interval=draw_interval, dt=dt, simulation_steps=simulation_steps)
            sim_visualizer.add_callback(partial(record_velocities, start_recording=0, end_recording=num_step-1))
            html = sim_visualizer.run_animation(num_step)
        else:
            sim_visualizer = SimulationVisualizer(simulator=simulator, particle_system=p, rad=rad, 
                                                  draw_interval=draw_interval, dt=dt, simulation_steps=simulation_steps, show=False)
            sim_visualizer.add_callback(partial(record_velocities, start_recording=0, end_recording=num_step-1))
            sim_visualizer.run_simulation(num_step)
            html = None
            
        # Convert recorded velocity data to shape (n_steps, n_particles, d)
        velocities_recorded = np.array(velocities_recorded)
        velocities_recorded = np.swapaxes(velocities_recorded, 1, 2)
        print(f"Experiment {exp+1}/{num_experiments} for method {method} completed. Recorded velocities shape: {velocities_recorded.shape}")
        
        # Calculate VACF
        vacf = compute_vacf(velocities_recorded, lags)
        vacf_norm = vacf / vacf[0]
        
        # Perform Fourier transform on VACF to obtain vibrational spectrum
        freq = np.fft.fftfreq(len(vacf), d=lags[1]-lags[0])
        S_omega = np.abs(np.fft.fft(vacf_norm))
        
        # Keep only the positive frequency part
        experiments_vacf.append((lags, vacf_norm))
        experiments_freq.append((freq[freq >= 0], S_omega[freq >= 0]))
        
    return experiments_vacf, experiments_freq, html


In [ ]:
num_particles, init_particles_positions = generate_circle_cluster(8, 0, 0, 3, True)
partical_info = {
    'num_particles': num_particles,
    'init_positions': init_particles_positions
}

langevin_params = {
    "method": "Langevin",
    "gamma": 1.0,
    "temperature": 0.3
}

nose_hoover_params = {
    "method": "NoseHoover",
    "gamma": 1.0,
    "temperature": 0.3,
    "Q": 0.1,
    "eta": 0.0
}
num_step = 100000
num_frame = 50

sim_params = {
    "num_experiments": 3,
    "num_step": num_step,
    "dt": 0.001,
    "draw_interval": num_step // num_frame,
    "simulation_steps": 500,
    "epsilon": 1.0,
    "sigma": 1.0,
    "rad": 40.0,
    "vacf_lags": np.arange(0, 200, 2),
    "show": True
}

# run the simulation
langevin_vacf, langevin_freq, langevin_html = run_simulation(partical_info, langevin_params, sim_params)
nose_hoover_vacf, nose_hoover_freq, nose_hoover_html = run_simulation(partical_info, nose_hoover_params, sim_params)

In [ ]:
# Call the function to plot the results
plot_simulation_results(langevin_vacf, langevin_freq, 'Langevin')
plot_simulation_results(nose_hoover_vacf, nose_hoover_freq, 'Nose-Hoover')

In [ ]:
display(langevin_html)

In [ ]:
display(nose_hoover_html)

In [ ]:
num_particles, init_particles_positions = generate_circle_cluster(8, 0, 0, 3, False)

perturbed_positions = np.copy(init_particles_positions)
for i in range(len(init_particles_positions)):
    perturbed_positions[i] += np.random.normal(0, 1, 2)


partical_info_perturbed = {
    'num_particles': num_particles,
    'init_positions': perturbed_positions
}

# show the perturbed initial positions
plt.figure(figsize=(6, 6))
plt.scatter(init_particles_positions[:, 0], init_particles_positions[:, 1], label='Initial positions')
plt.scatter(perturbed_positions[:, 0], perturbed_positions[:, 1], label='Perturbed positions')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Initial positions')
plt.legend()

In [ ]:
# run the simulation
langevin_vacf, langevin_freq, langevin_html = run_simulation(partical_info_perturbed, langevin_params, sim_params)
nose_hoover_vacf, nose_hoover_freq, nose_hoover_html = run_simulation(partical_info_perturbed, nose_hoover_params, sim_params)

In [ ]:
display(langevin_html)

In [ ]:
display(nose_hoover_html)

In [ ]:
# Call the function to plot the results
plot_simulation_results(langevin_vacf, langevin_freq, 'Langevin')
plot_simulation_results(nose_hoover_vacf, nose_hoover_freq, 'Nose-Hoover')